Consider the evolution of a simple Hamiltonian into a problem-specific Hamiltonian where the optimal solution is 'encoded' in the ground state of the problem Hamiltonian $\displaystyle \hat{\mathcal{H}}_{\text{P}}$

$$\hat{\mathcal{H}}\left(s\right) = \left(1 - s\right)\hat{\mathcal{H}}_0 + s\underbrace{\left(\alpha\hat{\mathcal{H}}_{\text{A}} + \hat{\mathcal{H}}_{\text{B}}\right)}_{\hat{\mathcal{H}}_{\text{P}}},\qquad\hat{\mathcal{H}}_{\text{B}} = -\sum_{i = 1}^np_ix_i$$

for some $\displaystyle s = \frac{t}{T}$ where $T$ is the total computation time and

$$\hat{\mathcal{H}}_0 = -\sum_i\sigma_x^i$$

and

$$\hat{\mathcal{H}}_{\text{A}} = \left(1 - \sum_{j = 1}^cy_j\right)^2 + \left(\sum_{j = 1}^cjy_j - \sum_{i = 1}^nw_ix_i\right)^2,\qquad S\left(n\right) = O\left(n + c\right)$$

where $\displaystyle \alpha > \max\left(p_j\right)$


In [23]:
from qutip import Qobj, basis, expect, ket2dm, mesolve, qeye, sigmax, sigmaz, tensor
import numpy as np
from matplotlib import gridspec
import matplotlib.pyplot as plt

np.set_printoptions(formatter={"all": lambda x: "{:.4g}".format(x)})

In [ ]:
#   Tensor product such that i-th element is sigma_x and rest are identity
def tensor_sigmax(i: int, n: int) -> Qobj:
    ops = [qeye(2) for _ in range(n)]
    ops[i] = sigmax()
    return tensor(ops)


tensor_sigmax(0, 1)

In [ ]:
#   Tensor product such that i-th element is (I - sigma_z) / 2 and rest are identity
def tensor_sigmaz(i: int, n: int) -> Qobj:
    ops = [qeye(2) for _ in range(n)]
    ops[i] = (qeye(2) - sigmaz()) / 2
    return tensor(ops)


tensor_sigmaz(0, 1)

In [ ]:
#   Parameters to set
profits = np.array([10, 2])
weights = np.array([1, 2])
capacity = 2

#   Derived parameters
num_items = profits.shape[0]
alpha = np.max(profits) + 1

total_qubits = num_items + capacity
num_states = np.power(2, total_qubits)

print(
    f"Total qubits: {num_items} (no. of items) + {capacity} (ancillary qubits) = {total_qubits}"
)

In [ ]:
#   Initial Hamiltonian H_0
H_0 = -sum(tensor_sigmax(i, total_qubits) for i in range(total_qubits))

H_0

In [ ]:
H_A1 = (
    1
    - sum(
        tensor_sigmaz(j, total_qubits) for j in range(num_items, total_qubits)
    )
) ** 2  #   starting from the 1st ancillary qubit

H_A1

In [ ]:
H_A2 = (
    sum(
        j * tensor_sigmaz(j, total_qubits)
        for j in range(num_items, total_qubits)
    )
    - sum(weights[i] * tensor_sigmaz(i, total_qubits) for i in range(num_items))
) ** 2

H_A2

In [ ]:
H_A = H_A1 + H_A2

H_A

In [ ]:
H_B = -sum(profits[i] * tensor_sigmaz(i, total_qubits) for i in range(num_items))

H_B

In [ ]:
H_P = alpha * H_A + H_B

H_P

In [ ]:
eigenvalues, eigenstates = H_P.eigenstates()

eigenvalues

In [ ]:
#   Obtain the ground eigenstate of H_0
ground_eigenvalue, ground_eigenstates = H_0.groundstate()
psi0 = ground_eigenstates

print(f"Ground eigenvalue = {ground_eigenvalue}\nGround state:")
psi0

In [35]:
times = np.linspace(0, 1, 10000)


#   Time-dependent Hamiltonian function
def H(s):
    return (1 - s) * H_0 + s * H_P


result = mesolve(H, psi0, times, e_ops=[])

In [ ]:
def generate_basis_states(n):
    """Generates a list of all basis states for n qubits."""
    return [
        tensor([basis(2, (i >> m) % 2) for m in range(n - 1, -1, -1)])
        for i in range(num_states)
    ]


basis_states = generate_basis_states(total_qubits)
basis_matrix = np.hstack([basis_state.full() for basis_state in basis_states])

print("Basis states generated!\n")

In [ ]:
state_matrix = np.hstack([psi.full() for psi in result.states])

probabilities = np.power(
    np.abs(np.dot(basis_matrix.T.conj(), state_matrix)), 2
).T.tolist()

print("Probabilities computed!\n")

In [ ]:
final_probabilities = [
    (format(idx, f"0{total_qubits}b"), prob)
    for idx, prob in enumerate(probabilities[-1])
]

print("Final probabilities formatted!\n")

In [ ]:
subset_times = np.linspace(0, 1, 100)

hamiltonians = [H(time) for time in subset_times]

eigenvalues = [ht.eigenenergies() for ht in hamiltonians]

all_eigenvalues = np.array([ev[:5] for ev in eigenvalues])

print("Energy eigenvalues computed!")

In [40]:
# print("Final probabilities of each state:")
# for state_label, prob in final_probabilities:
#     print(f"| {state_label[0:num_items]} {state_label[num_items:]} >: {prob:.4f}")

In [ ]:
sorted_probabilities = sorted(final_probabilities, key=lambda x: x[1], reverse=True)

num_item_bits = num_items
num_ancillary_bits = total_qubits - num_items
modifier_bit = total_qubits - 1


def calculate_item_weight(state_label):
    """Calculate total weight from item bits."""
    return sum(
        weights[i] for i, bit in enumerate(state_label[:num_item_bits]) if bit == "1"
    )


def calculate_ancillary_weight(state_label):
    """Calculate total weight from ancillary bits."""
    weight_from_ancillary = -1
    for j in range(num_item_bits, total_qubits):
        if state_label[j] == "1":
            if weight_from_ancillary != -1:
                weight_from_ancillary = -99999
            weight_from_ancillary = j - num_item_bits + 1
    return weight_from_ancillary


def calculate_total_profit(state_label):
    """Calculate total profit from item bits."""
    return sum(
        profits[i] for i, bit in enumerate(state_label[:num_items]) if bit == "1"
    )


print("States with the highest probabilities:")
for state_label, prob in sorted_probabilities[:5]:
    item_weight = calculate_item_weight(state_label)
    ancillary_weight = calculate_ancillary_weight(state_label)
    total_profit = calculate_total_profit(state_label)
    print(
        f"| {state_label[0:num_items]} {state_label[num_items:]} >: Probability = {prob:.4f}, "
        f"Item Weight = {item_weight}, Ancillary Weight = {ancillary_weight}, Total Profit = {total_profit}"
    )

In [ ]:
table_data = []
for state_label, prob in sorted_probabilities[:5]:
    item_weight = calculate_item_weight(state_label)
    ancillary_weight = calculate_ancillary_weight(state_label)
    total_profit = calculate_total_profit(state_label)
    table_data.append(
        [
            f"| {state_label[:num_items]} {state_label[num_items:]} >",
            f"{prob:.4f}",
            item_weight,
            ancillary_weight,
            total_profit,
        ]
    )


fig, ax = plt.subplots(figsize=(10, 2), dpi=300)
ax.axis("tight")
ax.axis("off")
ax.set(title="Top 5 states sorted by probability in descending order")
table = ax.table(
    cellText=table_data,
    colLabels=[
        "State",
        "Probability",
        "Item Weight",
        "Ancillary Weight",
        "Total Profit",
    ],
    cellLoc="center",
    loc="center",
)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)

for (i, j), cell in table.get_celld().items():
    if i == 0:
        cell.set_fontsize(12)
        cell.set_facecolor("#40466e")
        cell.set_text_props(color="w")

plt.show()

In [ ]:
filtered_states = []
total_probability = 0

for state_label, prob in final_probabilities:
    item_weight = calculate_item_weight(state_label)
    ancillary_weight = calculate_ancillary_weight(state_label)
    total_profit = calculate_total_profit(state_label)
    if item_weight == ancillary_weight:
        filtered_states.append(
            (state_label, prob, item_weight, ancillary_weight, total_profit)
        )
        total_probability += prob

#   Normalize the probabilities of remaining states
normalized_states = [
    (label, prob / total_probability, item_weight, anc_weight, tot_profit)
    for label, prob, item_weight, anc_weight, tot_profit in filtered_states
]

sorted_normalized_states = sorted(normalized_states, key=lambda x: x[1], reverse=True)

fig, ax = plt.subplots(figsize=(12, 2), dpi=300)
ax.axis("tight")
ax.axis("off")
ax.set(title=r"Filtered states (Item Weight $=$ Ancillary Weight)")
table_data = [
    [
        f"| {state[0][:num_items]} {state[0][num_items:]} >",
        f"{state[1]:.4f}",
        state[2],
        state[3],
        state[4],
    ]
    for state in sorted_normalized_states
]
table = ax.table(
    cellText=table_data,
    colLabels=[
        "Filtered State",
        "Probability",
        "Item Weight",
        "Ancillary Weight",
        "Total Profit",
    ],
    cellLoc="center",
    loc="center",
)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)

for (i, j), cell in table.get_celld().items():
    if i == 0:
        cell.set_fontsize(12)
        cell.set_facecolor("#40466e")
        cell.set_text_props(color="w")
    elif i > 0:
        item_weight = float(table_data[i - 1][2])
        if item_weight > capacity:
            for k in range(len(table_data[i - 1])):
                table[(i, k)].set_facecolor("#ffcccc")

plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 7), dpi=300)
gs = gridspec.GridSpec(2, 2, height_ratios=[5, 1])

ax1 = fig.add_subplot(gs[0, 0])

for idx in range(num_states):
    ax1.plot(
        times,
        [prob[idx] for prob in probabilities],
        label=f'|{format(idx, "0" + str(total_qubits) + "b")}>',
    )
    ax1.text(
        times[-1],
        probabilities[-1][idx],
        f'|{format(idx, "0" + str(total_qubits) + "b")}>',
        fontsize=8,
        horizontalalignment='center',
        verticalalignment='bottom'
    )

ax1.set(
    xlabel="Time units",
    ylabel="Probability",
    title="Probabilities of states against time",
)
ax1.grid(True, alpha=0.25)

ax3 = fig.add_subplot(gs[1, 0])

ax3.semilogy(times, [1 - sum(prob) for prob in probabilities], ls="", marker=".", ms=1)

ax3.set(
    xlabel="Time units",
    ylabel=r"$1 - \sum P$",
    title=r"Total probability difference from $1$",
)
ax3.grid(True, alpha=0.25)

ax2 = fig.add_subplot(gs[:, 1])

for idx in range(all_eigenvalues.shape[1]):
    ax2.plot(subset_times, all_eigenvalues[:, idx])

ax2.set(
    xlabel="Time units",
    ylabel="Energy eigenvalues",
    title="Evolution of energy eigenvalues against time",
)
ax2.grid(True, alpha=0.25)

plt.tight_layout()

plt.show()